In [1]:
'''
对叉车的搬运的工时进行统计
'''
import re
import os
import easygui
import numpy as np
import pandas as pd

YUANGONGS =  ['刘革红', '黄康', '吴长江', '李城', '胡国华', '代朝威']

In [9]:
def chuliRuChu(fname):
    # fname = r"F:\a00nutstore\006\zw\产成品出入库工作记录\仓库日常入库工作记录.xlsx"
    df = pd.read_excel(fname,header = 1)
    df = df.rename(columns = {'货物数量（件）':'jian','货物件数':'jian','日期':'date','车间':'address','单位名称':'address'})
    df = df[[i  for i  in  df.columns.to_list() if (i != '货物数量(托)') and  (i != '序号') and ('Unnamed' not in i)]]
    #删除空行
    df = df[df['jian'].notna()]
    df = df.set_index(['date','address','jian'])
    df = df.stack()
    df = df.reset_index()
    df.columns = ['date','chejian','jian','gongzhong','people']
    df = df[df.people.isin(YUANGONGS)]
    df.gongzhong = df.gongzhong.str.replace('\d+','',regex = True)
    data = []
    gp = df.groupby(['date','chejian','jian','gongzhong'])
    for k,v in gp:
        v = v.assign(jian2 = v.jian/len(v))
        data.append(v)
    df5 = pd.concat(data)
    #原表
    df5.to_excel('叉车搬运工时加工后原表01.xlsx',index = False)
    df6 = df5.assign(fapei = df5.gongzhong.str[:2],gongzhong1 = df5.gongzhong.str[2:])
    df6 = df6[[ 'fapei',
       'gongzhong1','people', 'jian2']]
    pivot = pd.pivot_table(df6,index = ['fapei','gongzhong1','people'],aggfunc = sum)
    pivot1 = pd.pivot_table(df6,index = 'people',columns = ['fapei','gongzhong1'],aggfunc = sum)
    return df6,pivot1
        


In [17]:
#入库
fname = r"F:\a00nutstore\006\zw\产成品出入库工作记录\仓库日常入库工作记录.xlsx"
df_ruku,pivot_ruku = chuliRuChu(fname)
pivot_ruku

C:\Users\redda\AppData\Local\Temp\ipykernel_21756\392583271.py:25: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = pd.pivot_table(df6,index = ['fapei','gongzhong1','people'],aggfunc = sum)
C:\Users\redda\AppData\Local\Temp\ipykernel_21756\392583271.py:26: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot1 = pd.pivot_table(df6,index = 'people',columns = ['fapei','gongzhong1'],aggfunc = sum)


jian2             
fapei                入库             
gongzhong1           叉车           搬运
people                              
代朝威                 NaN  2228.333333
刘革红         5140.333333          NaN
吴长江                 NaN  3877.333333
李城                  NaN  3420.333333
胡国华         3109.333333          NaN
黄康          4506.333333          NaN

In [11]:
#出库
fname = r"F:\a00nutstore\006\zw\产成品出入库工作记录\仓库日常出库工作记录.xlsx"
df_chuku,pivot_chuku = chuliRuChu(fname)
pivot_chuku

C:\Users\redda\AppData\Local\Temp\ipykernel_21756\392583271.py:25: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = pd.pivot_table(df6,index = ['fapei','gongzhong1','people'],aggfunc = sum)
C:\Users\redda\AppData\Local\Temp\ipykernel_21756\392583271.py:26: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot1 = pd.pivot_table(df6,index = 'people',columns = ['fapei','gongzhong1'],aggfunc = sum)


jian2                                       
fapei           发货                        配货             
gongzhong1      叉车           搬运           叉车           搬运
people                                                   
代朝威            NaN  9573.166667          NaN  7411.166667
刘革红         5026.0          NaN  4121.750000          NaN
吴长江            NaN  4435.166667          NaN  4543.250000
李城             NaN  5084.666667   214.500000  7138.583333
胡国华         9550.0          NaN  7173.166667          NaN
黄康          4514.0          NaN  7580.583333          NaN

In [12]:
df_ruku

,fapei,gongzhong1,people,jian2
6,入库,叉车,黄康,7.000000
7,搬运,,李城,7.000000
4,入库,叉车,胡国华,120.000000
5,搬运,,代朝威,120.000000
0,入库,叉车,刘革红,433.000000
...,...,...,...,...
127,入库,叉车,黄康,256.333333
128,入库,叉车,胡国华,256.333333
129,搬运,,吴长江,256.333333
130,搬运,,李城,256.333333


In [13]:
df_chuku

,fapei,gongzhong1,people,jian2
19,发货,叉车,胡国华,155.0
20,发货,搬运,代朝威,155.0
21,配货,叉车,胡国华,77.5
22,配货,叉车,刘革红,77.5
23,配货,搬运,代朝威,155.0
...,...,...,...,...
648,配货,搬运,李城,62.0
649,发货,叉车,胡国华,10.0
650,发货,搬运,吴长江,10.0
651,配货,叉车,胡国华,10.0


In [29]:
total = pd.concat([df_ruku,df_chuku])
total = total.rename(columns = {'fapei':'项目','gongzhong1':'工种','people':'人员','jian2':'件数'})
total = total.astype({'件数':'int'})
total 

,项目,工种,人员,件数
6,入库,叉车,黄康,7
7,入库,搬运,李城,7
4,入库,叉车,胡国华,120
5,入库,搬运,代朝威,120
0,入库,叉车,刘革红,433
...,...,...,...,...
648,配货,搬运,李城,62
649,发货,叉车,胡国华,10
650,发货,搬运,吴长江,10
651,配货,叉车,胡国华,10


In [31]:
pivot= pd.pivot_table(total,index = '人员',columns = ['项目','工种'],aggfunc = sum,margins = True,margins_name = '小计')
pivot

C:\Users\redda\AppData\Local\Temp\ipykernel_21756\3878086793.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot= pd.pivot_table(total,index = '人员',columns = ['项目','工种'],aggfunc = sum,margins = True,margins_name = '小计')
C:\Users\redda\AppData\Local\Temp\ipykernel_21756\3878086793.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot= pd.pivot_table(total,index = '人员',columns = ['项目','工种'],aggfunc = sum,margins = True,margins_name = '小计')
C:\Users\redda\AppData\Local\Temp\ipykernel_21756\3878086793.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum.

件数                                                   
项目        入库               发货                配货              小计
工种        叉车      搬运       叉车       搬运       叉车       搬运       
人员                                                             
代朝威      NaN  2224.0      NaN   9567.0      NaN   7405.0  19196
刘革红   5134.0     NaN   5026.0      NaN   4113.0      NaN  14273
吴长江      NaN  3872.0      NaN   4430.0      NaN   4538.0  12840
李城       NaN  3416.0      NaN   5078.0    214.0   7131.0  15839
胡国华   3104.0     NaN   9549.0      NaN   7164.0      NaN  19817
黄康    4501.0     NaN   4514.0      NaN   7570.0      NaN  16585
小计   12739.0  9512.0  19089.0  19075.0  19061.0  19074.0  98550

In [32]:
pivot1 = pivot.astype('int')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [33]:
pivot.to_excel('叉车搬运工时汇总2.xlsx')